# Text-to-SQL with Vanna.ai - Complete Tutorial

**Duration:** 1-2 hours  
**Level:** Intermediate (requires SQL knowledge and basic LLM understanding)  
**Framework:** Vanna.ai 2.0 Agent Framework  
**Database:** PostgreSQL (Supabase)

---

## What You'll Learn

1. What Text-to-SQL is and why it matters
2. How to set up Vanna.ai 2.0 with OpenAI and PostgreSQL
3. Understanding the Agent framework architecture
4. Generating SQL from natural language questions
5. Executing queries and handling results
6. Best practices for production deployment

---

## Prerequisites

- OpenAI API key
- PostgreSQL database (Supabase or local)
- Python 3.12+
- Basic understanding of SQL and LLMs

---

# Section 1: Introduction to Text-to-SQL

## What is Text-to-SQL?

Text-to-SQL converts natural language questions into SQL queries:

```
Question: "How many customers do we have?"
         ↓
SQL: SELECT COUNT(*) FROM customers
         ↓
Result: 100
```

## Why Use Text-to-SQL?

1. **Democratize Data Access** - Non-technical users can query databases
2. **Faster Analytics** - No manual SQL writing required
3. **Business Intelligence** - Power chatbots and dashboards
4. **Real-World Use Cases**: Slack bots, analytics dashboards, customer support tools

## Technical Architecture

```
Natural Language Question
         ↓
    LLM (OpenAI GPT-4o)
         ↓
    SQL Generation
         ↓
    PostgreSQL Database
         ↓
    Results (DataFrame)
```

## Vanna.ai 2.0 Overview

- **Agent Framework** - Modern architecture with user awareness
- **Modular Design** - Swap LLMs, databases, tools
- **Security** - User permissions and access control
- **Streaming UI** - Real-time component updates

---

# Section 2: Environment Setup

## Install Dependencies

In [ ]:
# Install required packages
#!pip install vanna openai psycopg2-binary python-dotenv pandas sqlalchemy -q

## Import Libraries

**Important:** Vanna 2.0 uses different imports than Legacy (0.x)

In [1]:
# Core Vanna 2.0 imports
from vanna import Agent
from vanna.integrations.openai import OpenAILlmService
from vanna.integrations.postgres import PostgresRunner
from vanna.core.registry import ToolRegistry
from vanna.tools import RunSqlTool
from vanna.core.user import UserResolver, User, RequestContext
from vanna.integrations.local.agent_memory import DemoAgentMemory

# Standard libraries
import os
import pandas as pd
import psycopg2
from dotenv import load_dotenv
from urllib.parse import urlparse
from IPython.display import display

print("✓ Libraries imported successfully")

✓ Libraries imported successfully


## Load Environment Variables

In [2]:
# Load from .env file
load_dotenv()

# Get credentials
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
DATABASE_URL = os.getenv("DATABASE_URL")

# Verify credentials exist
if not OPENAI_API_KEY:
    raise ValueError("OPENAI_API_KEY not found in environment")
if not DATABASE_URL:
    raise ValueError("DATABASE_URL not found in environment")

print("✓ Environment variables loaded")
print(f"  OpenAI API Key: {OPENAI_API_KEY[:20]}...")
print(f"  Database: {DATABASE_URL.split('@')[1] if '@' in DATABASE_URL else 'configured'}")

✓ Environment variables loaded
  OpenAI API Key: sk-proj-PNzG9iLCEqCC...
  Database: db.ixfxeloafnhmyeelchzc.supabase.co:5432/postgres


---

# Section 3: Initialize Vanna 2.0 Agent

## Architecture Components

Vanna 2.0 uses a modular Agent architecture:

1. **LlmService** - The language model (OpenAI GPT-4o)
2. **SqlRunner** - Database connection and execution
3. **ToolRegistry** - Available tools for the Agent
4. **UserResolver** - User authentication and permissions
5. **AgentMemory** - Conversation history
6. **Agent** - Orchestrates everything

## Step 1: Initialize LLM Service

In [3]:
# Initialize OpenAI GPT-4o
llm = OpenAILlmService(
    api_key=OPENAI_API_KEY,
    model="gpt-4o"  # Latest OpenAI model
)

print("✓ LLM Service initialized (OpenAI GPT-4o)")

✓ LLM Service initialized (OpenAI GPT-4o)


## Step 2: Initialize PostgreSQL Runner

**Important:** Use `connection_string` approach (simplest)

In [4]:
# Connect to PostgreSQL using connection string
postgres_runner = PostgresRunner(
    connection_string=DATABASE_URL
)

print("✓ PostgreSQL Runner initialized")

✓ PostgreSQL Runner initialized


## Step 3: Register Tools

Tools define what the Agent can do. We'll register the `RunSqlTool`.

In [5]:
# Create tool registry
tools = ToolRegistry()

# Register SQL execution tool
tools.register_local_tool(
    RunSqlTool(sql_runner=postgres_runner),
    access_groups=['user', 'admin']  # Who can use this tool
)

print("✓ Tools registered (RunSqlTool)")

✓ Tools registered (RunSqlTool)


## Step 4: Create User Resolver

User resolver handles authentication and permissions.

In [6]:
class SimpleUserResolver(UserResolver):
    """Simple user resolver for tutorial purposes"""
    
    async def resolve_user(self, request_context: RequestContext) -> User:
        return User(
            id="tutorial_user",
            email="student@tutorial.com",
            group_memberships=['user', 'admin']  # Full access
        )

user_resolver = SimpleUserResolver()

print("✓ User Resolver created")

✓ User Resolver created


## Step 5: Initialize Agent

The Agent orchestrates all components.

In [8]:
# Create the Agent
agent = Agent(
    llm_service=llm,
    tool_registry=tools,
    user_resolver=user_resolver,
    agent_memory=DemoAgentMemory()  # In-memory conversation history
)

print("✓ Agent initialized successfully!")
print("\n=== Vanna 2.0 Agent Ready ===")

✓ Agent initialized successfully!

=== Vanna 2.0 Agent Ready ===


---

# Section 4: Understanding the Database

## Database Schema Overview

Our e-commerce database has 3 tables:

### 1. `customers` (100 rows)
- `id` - Primary key
- `name` - Customer name
- `email` - Email address
- `segment` - SMB, Enterprise, or Individual
- `country` - Customer country

### 2. `products` (50 rows)
- `id` - Primary key
- `name` - Product name
- `category` - Product category
- `price` - Product price
- `stock_quantity` - Inventory count

### 3. `orders` (200 rows)
- `id` - Primary key
- `customer_id` - Foreign key to customers
- `order_date` - Order date
- `total_amount` - Order total (NOT 'price'!)
- `status` - Pending, Delivered, Cancelled, Processing

### Relationships
- `customers.id` → `orders.customer_id` (one-to-many)

## Create Helper Function for Direct SQL

Sometimes we want to run SQL directly without the Agent.

In [9]:
def run_sql_simple(sql: str) -> pd.DataFrame:
    """
    Execute SQL directly using psycopg2 (non-async, simple).
    
    Args:
        sql: SQL query string
        
    Returns:
        DataFrame with results
    """
    # Parse DATABASE_URL
    parsed = urlparse(DATABASE_URL)
    
    # Connect to database
    conn = psycopg2.connect(
        host=parsed.hostname,
        database=parsed.path[1:],  # Remove leading '/'
        user=parsed.username,
        password=parsed.password,
        port=parsed.port or 5432
    )
    
    # Execute query
    df = pd.read_sql_query(sql, conn)
    conn.close()
    
    return df

print("✓ Helper function created: run_sql_simple()")

✓ Helper function created: run_sql_simple()


## Preview Database Tables

In [10]:
# Preview customers table
print("=== CUSTOMERS TABLE (Sample) ===")
customers_sample = run_sql_simple("SELECT * FROM customers LIMIT 5")
display(customers_sample)

=== CUSTOMERS TABLE (Sample) ===


/var/folders/cj/13vbmk7n7fqgmdnqjjwn1bx80000gn/T/ipykernel_13249/4022728488.py:24: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql_query(sql, conn)


,id,name,email,segment,country,created_at,updated_at
0,1,Craig Robinson,cescobar@example.net,SMB,USA,2026-01-07 06:47:51.713204,2026-01-07 06:47:51.713204
1,2,Pamela Holt,taylordavid@example.net,Individual,France,2026-01-07 06:47:51.713204,2026-01-07 06:47:51.713204
2,3,Vanessa Johnson,dunncorey@example.com,Enterprise,Australia,2026-01-07 06:47:51.713204,2026-01-07 06:47:51.713204
3,4,Kimberly Cook,waltersmelissa@example.net,Individual,France,2026-01-07 06:47:51.713204,2026-01-07 06:47:51.713204
4,5,Colleen Miller,dhunt@example.net,SMB,Germany,2026-01-07 06:47:51.713204,2026-01-07 06:47:51.713204


In [11]:
# Preview products table
print("=== PRODUCTS TABLE (Sample) ===")
products_sample = run_sql_simple("SELECT * FROM products LIMIT 5")
display(products_sample)

=== PRODUCTS TABLE (Sample) ===


/var/folders/cj/13vbmk7n7fqgmdnqjjwn1bx80000gn/T/ipykernel_13249/4022728488.py:24: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql_query(sql, conn)


,id,name,category,price,stock_quantity,description,created_at,updated_at
0,1,Support Plan,Services,1277.07,260,None,2026-01-07 06:47:51.713204,2026-01-07 06:47:51.713204
1,2,Consulting Carter-Lewis,Services,1078.47,128,Laugh listen pass keep process. With crime res...,2026-01-07 06:47:51.713204,2026-01-07 06:47:51.713204
2,3,Analytics Platform,Software,41.09,245,Read effect call summer. Tree field become sev...,2026-01-07 06:47:51.713204,2026-01-07 06:47:51.713204
3,4,Support Plan,Services,1801.58,497,None,2026-01-07 06:47:51.713204,2026-01-07 06:47:51.713204
4,5,"Project Management Tool Clark, Lloyd and Miller",Software,152.28,262,Program common moment see.\nSuffer suffer son ...,2026-01-07 06:47:51.713204,2026-01-07 06:47:51.713204


In [12]:
# Preview orders table
print("=== ORDERS TABLE (Sample) ===")
orders_sample = run_sql_simple("SELECT * FROM orders LIMIT 5")
display(orders_sample)

=== ORDERS TABLE (Sample) ===


/var/folders/cj/13vbmk7n7fqgmdnqjjwn1bx80000gn/T/ipykernel_13249/4022728488.py:24: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql_query(sql, conn)


,id,customer_id,order_date,total_amount,status,shipping_address,created_at,updated_at
0,1,2,2025-04-12,4664.19,Delivered,Unit 0084 Box 8448\nDPO AE 03858,2026-01-07 06:47:51.713204,2026-01-07 06:47:51.713204
1,2,20,2025-01-28,2431.64,Pending,"544 Ochoa Haven Suite 000\nNorth Donaldfort, C...",2026-01-07 06:47:51.713204,2026-01-07 06:47:51.713204
2,3,94,2025-05-09,1096.18,Delivered,"51041 Margaret Stream\nSouth Erica, RI 61867",2026-01-07 06:47:51.713204,2026-01-07 06:47:51.713204
3,4,79,2025-06-01,4321.68,Delivered,"94942 Mary Lodge Suite 686\nYolandaburgh, TN 8...",2026-01-07 06:47:51.713204,2026-01-07 06:47:51.713204
4,5,67,2025-12-17,738.59,Delivered,"PSC 6888, Box 2931\nAPO AA 67668",2026-01-07 06:47:51.713204,2026-01-07 06:47:51.713204


## Database Statistics

In [13]:
# Get table counts
stats = run_sql_simple("""
SELECT 
    (SELECT COUNT(*) FROM customers) as total_customers,
    (SELECT COUNT(*) FROM products) as total_products,
    (SELECT COUNT(*) FROM orders) as total_orders,
    (SELECT COUNT(DISTINCT segment) FROM customers) as customer_segments,
    (SELECT COUNT(DISTINCT category) FROM products) as product_categories,
    (SELECT COUNT(DISTINCT status) FROM orders) as order_statuses
""")

print("=== DATABASE STATISTICS ===")
display(stats)

=== DATABASE STATISTICS ===


/var/folders/cj/13vbmk7n7fqgmdnqjjwn1bx80000gn/T/ipykernel_13249/4022728488.py:24: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql_query(sql, conn)


,total_customers,total_products,total_orders,customer_segments,product_categories,order_statuses
0,100,50,200,3,6,4


---

# Section 5: Provide Schema Documentation to Agent

## Why Schema Documentation Matters

The Agent needs to understand:
- What tables exist
- What columns each table has
- Data types and relationships
- Business terminology

**Without schema knowledge, the Agent will generate incorrect SQL!**

Example: It might look for a `price` column in `orders` when the correct column is `total_amount`.

## Provide Schema Context

We'll give the Agent a detailed schema description.

In [14]:
# Schema documentation
SCHEMA_CONTEXT = """
DATABASE SCHEMA:

Table: customers
Columns:
  - id (SERIAL PRIMARY KEY)
  - name (VARCHAR) - Customer full name
  - email (VARCHAR) - Customer email address
  - segment (VARCHAR) - One of: 'SMB', 'Enterprise', 'Individual'
  - country (VARCHAR) - Customer country
  - created_at (TIMESTAMP)
  - updated_at (TIMESTAMP)

Table: products
Columns:
  - id (SERIAL PRIMARY KEY)
  - name (VARCHAR) - Product name
  - category (VARCHAR) - Product category (Electronics, Software, Hardware, etc.)
  - price (DECIMAL) - Product unit price
  - stock_quantity (INT) - Current inventory count
  - description (TEXT)
  - created_at (TIMESTAMP)
  - updated_at (TIMESTAMP)

Table: orders
Columns:
  - id (SERIAL PRIMARY KEY)
  - customer_id (INT) - Foreign key to customers.id
  - order_date (DATE) - Date of order
  - total_amount (DECIMAL) - TOTAL ORDER PRICE (use this for revenue, NOT 'price'!)
  - status (VARCHAR) - One of: 'Pending', 'Delivered', 'Cancelled', 'Processing'
  - shipping_address (TEXT)
  - created_at (TIMESTAMP)
  - updated_at (TIMESTAMP)

RELATIONSHIPS:
  - customers.id → orders.customer_id (one-to-many)
  
IMPORTANT NOTES:
  - For order revenue/pricing, use orders.total_amount (NOT 'price')
  - Customer segments: 'SMB', 'Enterprise', 'Individual' (case-sensitive)
  - Order statuses: 'Pending', 'Delivered', 'Cancelled', 'Processing' (case-sensitive)
  - To join customers and orders: JOIN orders ON customers.id = orders.customer_id
"""

print("✓ Schema context prepared")
print("\nThis will help the Agent understand the database structure.")

✓ Schema context prepared

This will help the Agent understand the database structure.


---

# Section 6: Querying with the Agent

## Create Helper Function for Agent Queries

**Important:** Vanna 2.0's `agent.send_message()` returns an **async generator**, not a simple awaitable.

We must use `async for` to iterate through UI components.

In [15]:
# Create a request context (simulates HTTP request)
request_context = RequestContext()

# Test query
print("🧪 Testing Agent with: 'How many customers are in the database?'\n")

try:
    # Iterate over streaming results
    async for ui_component in agent.send_message(
        request_context=request_context,
        message="How many customers are in the database?"
    ):
        # Each ui_component is a piece of the response
          print(ui_component)

    print("\n✅ Agent responded successfully!")

except Exception as e:
      print(f"❌ Error: {e}")
      print("\nTroubleshooting:")
      print("  • Make sure DATABASE_URL is set correctly")
      print("  • Ensure the database has a 'customers' table")
      print("  • Check OPENAI_API_KEY is valid")

🧪 Testing Agent with: 'How many customers are in the database?'

timestamp='2026-01-18T04:36:31.166166' rich_component=StatusBarUpdateComponent(id='vanna-status-bar', type=<ComponentType.STATUS_BAR_UPDATE: 'status_bar_update'>, lifecycle=<ComponentLifecycle.CREATE: 'create'>, data={}, children=[], timestamp='2026-01-18T04:36:31.166127', visible=True, interactive=False, status='working', message='Processing your request...', detail='Analyzing query') simple_component=None
timestamp='2026-01-18T04:36:31.166415' rich_component=TaskTrackerUpdateComponent(id='vanna-task-tracker', type=<ComponentType.TASK_TRACKER_UPDATE: 'task_tracker_update'>, lifecycle=<ComponentLifecycle.CREATE: 'create'>, data={}, children=[], timestamp='2026-01-18T04:36:31.166407', visible=True, interactive=False, operation=<TaskOperation.ADD_TASK: 'add_task'>, task=Task(id='213a1f0d-8e16-45bb-a0bf-83c45693e301', title='Load conversation context', description='Reading message history and user context', status='pending',

In [26]:
async def ask_agent(question: str, include_schema: bool = True):
    """
    Ask the Agent a question and display results.
    
    Args:
        question: Natural language question
        include_schema: Whether to include schema context (recommended: True)
    """
    # Prepend schema context to question
    if include_schema:
        full_message = f"{SCHEMA_CONTEXT}\n\nQUESTION: {question}"
    else:
        full_message = question
    
    # Create request context
    request_context = RequestContext()
    
    print(f"🤔 Question: {question}\n")
    
    # Send message to agent (returns async generator)
    async for component in agent.send_message(
        request_context=request_context,
        message=full_message
    ):
        # Extract the UI component
        rich_comp = component.rich_component
        
        # Handle different component types
        if hasattr(rich_comp, 'rows') and rich_comp.rows:
            # DataFrameComponent - extract data from 'rows' attribute
            df = pd.DataFrame(rich_comp.rows)
            print("📊 Results:")
            display(df)
            print()
        
        elif hasattr(rich_comp, 'text') and rich_comp.text:
            # RichTextComponent - display text
            print(f"💬 {rich_comp.text}\n")
        
        elif hasattr(rich_comp, 'sql') and rich_comp.sql:
            # SQL code component
            print(f"🔍 Generated SQL:\n{rich_comp.sql}\n")

print("✓ Helper function created: ask_agent()")

✓ Helper function created: ask_agent()


### With Explanations

In [ ]:
# async def ask_agent(question: str):
#       """Ask agent and display results properly."""
#       print(f"❓ Question: {question}\n")

#       request_context = RequestContext()

#       print("="*80 + "\n")

#       async for component in agent.send_message(
#           request_context=request_context,
#           message=question
#       ):
#           rich_comp = component.rich_component

#           # Check for DataFrame component
#           if hasattr(rich_comp, 'type') and 'DATAFRAME' in str(rich_comp.type):
#               # ✅ Get data from 'rows' attribute!
#               if hasattr(rich_comp, 'rows') and rich_comp.rows:
#                   df = pd.DataFrame(rich_comp.rows)

#                   print("📊 Query Results:\n")
#                   from IPython.display import display
#                   display(df)
#                   print()

#           # Check for text explanation
#           elif hasattr(rich_comp, 'type') and 'TEXT' in str(rich_comp.type):
#               if hasattr(rich_comp, 'content') and rich_comp.content:
#                   print("💬 Explanation:\n")
#                   print(rich_comp.content)
#                   print()

#       print("="*80 + "\n")

## Example 1: Simple COUNT Query

In [19]:
await ask_agent("How many customers do we have?")

❓ Question: How many customers do we have?


📊 Query Results:



,count
0,100



💬 Explanation:

We have a total of 100 customers in our database.




## Example 2: Aggregation Query

In [20]:
await ask_agent("What is the total revenue from all orders?")

❓ Question: What is the total revenue from all orders?


📊 Query Results:



,table_catalog,table_schema,table_name,column_name,ordinal_position,column_default,is_nullable,data_type,character_maximum_length,character_octet_length,...,is_identity,identity_generation,identity_start,identity_increment,identity_maximum,identity_minimum,identity_cycle,is_generated,generation_expression,is_updatable
0,postgres,public,orders,id,1,nextval('orders_id_seq'::regclass),NO,integer,NaN,NaN,...,NO,None,None,None,None,None,NO,NEVER,None,YES
1,postgres,public,orders,customer_id,2,None,NO,integer,NaN,NaN,...,NO,None,None,None,None,None,NO,NEVER,None,YES
2,postgres,public,orders,order_date,3,CURRENT_DATE,NO,date,NaN,NaN,...,NO,None,None,None,None,None,NO,NEVER,None,YES
3,postgres,public,orders,total_amount,4,None,NO,numeric,NaN,NaN,...,NO,None,None,None,None,None,NO,NEVER,None,YES
4,postgres,public,orders,status,5,None,YES,character varying,20.0,8.000000e+01,...,NO,None,None,None,None,None,NO,NEVER,None,YES
5,postgres,public,orders,shipping_address,6,None,YES,text,NaN,1.073742e+09,...,NO,None,None,None,None,None,NO,NEVER,None,YES
6,postgres,public,orders,created_at,7,CURRENT_TIMESTAMP,YES,timestamp without time zone,NaN,NaN,...,NO,None,None,None,None,None,NO,NEVER,None,YES
7,postgres,public,orders,updated_at,8,CURRENT_TIMESTAMP,YES,timestamp without time zone,NaN,NaN,...,NO,None,None,None,None,None,NO,NEVER,None,YES



📊 Query Results:



,total_revenue
0,467204.56



💬 Explanation:

The total revenue from all orders is $467,204.56.




## Example 3: Filtering Query

In [21]:
await ask_agent("How many orders have been delivered?")

❓ Question: How many orders have been delivered?


📊 Query Results:



,delivered_orders
0,140



💬 Explanation:

There have been a total of 140 orders that have been delivered.




## Example 4: JOIN Query

In [22]:
await ask_agent("Show me the top 5 customers by total spending")

❓ Question: Show me the top 5 customers by total spending


📊 Query Results:



,customer_id,total_spending
0,65,15628.40
1,26,15337.06
2,46,13940.26
3,54,13323.72
4,7,13311.52



💬 Explanation:

Here are the top 5 customers by total spending:

1. Customer ID 65: $15,628.40
2. Customer ID 26: $15,337.06
3. Customer ID 46: $13,940.26
4. Customer ID 54: $13,323.72
5. Customer ID 7: $13,311.52

These customers have the highest total spending amounts in the database.




## Example 5: GROUP BY with JOIN

In [27]:
await ask_agent("What is the average order value for each customer segment?")

🤔 Question: What is the average order value for each customer segment?

📊 Results:


,segment,average_order_value
0,Enterprise,2105.0842222222222222
1,Individual,2466.4253623188405797
2,SMB,2352.2374418604651163


## Example 6: Time-Based Filtering

In [24]:
await ask_agent("Show orders from the last 30 days")

❓ Question: Show orders from the last 30 days


📊 Query Results:



,id,customer_id,order_date,total_amount,status,shipping_address,created_at,updated_at
0,22,54,2026-01-04,3762.70,Delivered,"7247 Sarah Light\nMorrisfort, ID 99759",2026-01-07 06:47:51.713204,2026-01-07 06:47:51.713204
1,42,83,2025-12-25,2692.29,Delivered,"661 Jacob Grove\nLake Jesus, OH 83145",2026-01-07 06:47:51.713204,2026-01-07 06:47:51.713204
2,95,13,2025-12-24,3410.28,Cancelled,USNS Peters\nFPO AA 25044,2026-01-07 06:47:51.713204,2026-01-07 06:47:51.713204
3,128,97,2026-01-02,3080.52,Delivered,USNS Hanson\nFPO AP 10571,2026-01-07 06:47:51.713204,2026-01-07 06:47:51.713204
4,153,33,2025-12-23,4077.61,Cancelled,"095 Susan Harbor\nWest Justinborough, CO 43673",2026-01-07 06:47:51.713204,2026-01-07 06:47:51.713204
5,200,26,2025-12-19,2229.04,Processing,"156 Jennifer Mall Apt. 926\nLake Matthewside, ...",2026-01-07 06:47:51.713204,2026-01-07 06:47:51.713204



💬 Explanation:

Here are the orders from the last 30 days:

1. **Order #22**
   - Date: 2026-01-04
   - Amount: $3,762.70
   - Status: Delivered
   - Shipping Address: 7247 Sarah Light, Morrisfort, ID 99759

2. **Order #42**
   - Date: 2025-12-25
   - Amount: $2,692.29
   - Status: Delivered
   - Shipping Address: 661 Jacob Grove, Lake Jesus, OH 83145

3. **Order #95**
   - Date: 2025-12-24
   - Amount: $3,410.28
   - Status: Cancelled
   - Shipping Address: USNS Peters, FPO AA 25044

4. **Order #128**
   - Date: 2026-01-02
   - Amount: $3,080.52
   - Status: Delivered
   - Shipping Address: USNS Hanson, FPO AP 10571

5. **Order #153**
   - Date: 2025-12-23
   - Amount: $4,077.61
   - Status: Cancelled
   - Shipping Address: 095 Susan Harbor, West Justinborough, CO 43673

6. **Order #200**
   - Date: 2025-12-19
   - Amount: $2,229.04
   - Status: Processing
   - Shipping Address: 156 Jennifer Mall Apt. 926, Lake Matthewside, FM 17579

The results have also been saved to a CSV file nam

## Example 7: Complex Business Question

In [25]:
await ask_agent("What is the total revenue from delivered orders placed by Enterprise customers?")

❓ Question: What is the total revenue from delivered orders placed by Enterprise customers?


💬 Explanation:

It seems there was an error due to a column name issue. I'll adjust the query and retry.

💬 Explanation:

It appears that the column name for the order amount is not correct in the query. Let me review the structure and try again.

📊 Query Results:



,column_name
0,id
1,customer_id
2,order_date
3,total_amount
4,status
5,shipping_address
6,created_at
7,updated_at



💬 Explanation:

There seems to be an issue with the `customer_type` column not existing. Let me investigate the structure of this dataset further to identify the correct way to determine customer type.

📊 Query Results:



,column_name
0,id
1,name
2,email
3,segment
4,country
5,created_at
6,updated_at



💬 Explanation:

It looks like the customer type information might be stored in the `segment` column of the `customers` table. I will join the `orders` and `customers` tables to calculate the total revenue from delivered orders placed by Enterprise customers. Let me perform the query.

📊 Query Results:



,total_revenue
0,66412.87



💬 Explanation:

The total revenue from delivered orders placed by Enterprise customers is $66,412.87.




---

# Section 7: Understanding How the Agent Works

## The Agent's Workflow

When you ask a question, here's what happens:

1. **Question Processing** - Agent receives your natural language question
2. **Schema Retrieval** - Agent uses the schema context we provided
3. **SQL Generation** - GPT-4o generates SQL based on the question and schema
4. **Tool Execution** - Agent calls `RunSqlTool` to execute the SQL
5. **Result Formatting** - Results are returned as UI components (DataFrameComponent)
6. **Response** - We extract and display the results

## Why Schema Context is Critical

Compare these two approaches:

### Without Schema Context:

In [28]:
# This might generate incorrect SQL (looking for 'price' instead of 'total_amount')
await ask_agent(
    "What is the total revenue from delivered orders?",
    include_schema=False  # No schema context
)

🤔 Question: What is the total revenue from delivered orders?

📊 Results:


,table_catalog,table_schema,table_name,column_name,ordinal_position,column_default,is_nullable,data_type,character_maximum_length,character_octet_length,...,is_identity,identity_generation,identity_start,identity_increment,identity_maximum,identity_minimum,identity_cycle,is_generated,generation_expression,is_updatable
0,postgres,public,orders,id,1,nextval('orders_id_seq'::regclass),NO,integer,NaN,NaN,...,NO,None,None,None,None,None,NO,NEVER,None,YES
1,postgres,public,orders,customer_id,2,None,NO,integer,NaN,NaN,...,NO,None,None,None,None,None,NO,NEVER,None,YES
2,postgres,public,orders,order_date,3,CURRENT_DATE,NO,date,NaN,NaN,...,NO,None,None,None,None,None,NO,NEVER,None,YES
3,postgres,public,orders,total_amount,4,None,NO,numeric,NaN,NaN,...,NO,None,None,None,None,None,NO,NEVER,None,YES
4,postgres,public,orders,status,5,None,YES,character varying,20.0,8.000000e+01,...,NO,None,None,None,None,None,NO,NEVER,None,YES
5,postgres,public,orders,shipping_address,6,None,YES,text,NaN,1.073742e+09,...,NO,None,None,None,None,None,NO,NEVER,None,YES
6,postgres,public,orders,created_at,7,CURRENT_TIMESTAMP,YES,timestamp without time zone,NaN,NaN,...,NO,None,None,None,None,None,NO,NEVER,None,YES
7,postgres,public,orders,updated_at,8,CURRENT_TIMESTAMP,YES,timestamp without time zone,NaN,NaN,...,NO,None,None,None,None,None,NO,NEVER,None,YES



📊 Results:


,id,customer_id,order_date,total_amount,status,shipping_address,created_at,updated_at
0,1,2,2025-04-12,4664.19,Delivered,Unit 0084 Box 8448\nDPO AE 03858,2026-01-07 06:47:51.713204,2026-01-07 06:47:51.713204



📊 Results:


,total_revenue
0,346189.37


### With Schema Context:

In [29]:
# This will generate correct SQL (using 'total_amount')
await ask_agent(
    "What is the total revenue from delivered orders?",
    include_schema=True  # Schema context included
)

🤔 Question: What is the total revenue from delivered orders?

📊 Results:


,total_revenue
0,346189.37


## Verifying Agent Results

Let's verify the Agent's answer with direct SQL:

In [30]:
# Direct SQL query for verification
verification_result = run_sql_simple("""
SELECT 
    COUNT(*) as delivered_count,
    SUM(total_amount) as total_revenue
FROM orders
WHERE status = 'Delivered'
""")

print("=== VERIFICATION (Direct SQL) ===")
display(verification_result)

=== VERIFICATION (Direct SQL) ===


/var/folders/cj/13vbmk7n7fqgmdnqjjwn1bx80000gn/T/ipykernel_13249/4022728488.py:24: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql_query(sql, conn)


,delivered_count,total_revenue
0,140,346189.37


---

# Section 8: Testing Various Question Types

## Simple Queries

In [31]:
await ask_agent("How many products are in stock?")

🤔 Question: How many products are in stock?

📊 Results:


,total_in_stock
0,12964


## Aggregation Queries

In [32]:
await ask_agent("What is the average product price?")

🤔 Question: What is the average product price?

📊 Results:


,average_product_price
0,954.7542000000000000


In [28]:
await ask_agent("What is the highest order value?")

🤔 Question: What is the highest order value?

📊 Results:


,highest_order_value
0,4973.14


## Filtering and Sorting

In [33]:
await ask_agent("Show me the 10 most expensive products")

🤔 Question: Show me the 10 most expensive products

📊 Results:


,id,name,price
0,73,"Charger Smith, Diaz and Welch",1922.18
1,74,"Router Kim, Jimenez and Miller",1910.78
2,95,Training Brady-Hernandez,1847.37
3,16,"Laptop Zimmerman, Wilson and Holt",1841.93
4,33,"Case Phillips, Harris and Williams",1832.58
5,82,Storage Drive Mendez-Leon,1818.14
6,48,"Technical Manual Reed, Schmitt and Crane",1801.89
7,4,Support Plan,1801.58
8,99,Laptop Thompson-Navarro,1773.75
9,38,Maintenance Bush LLC,1724.00


In [30]:
await ask_agent("Which customers are from the USA?")

🤔 Question: Which customers are from the USA?

📊 Results:


,id,name,email,segment,country
0,1,Craig Robinson,cescobar@example.net,SMB,USA
1,10,Bryce Russell,timothymartin@example.net,Individual,USA
2,16,Rebecca Blevins,amandathomas@example.com,SMB,USA
3,21,Kathy Finley,robert43@example.org,Enterprise,USA
4,23,Megan Diaz,brittanyhouston@example.net,SMB,USA
5,27,Melanie King,qgrant@example.net,SMB,USA
6,30,Christopher Sparks,ymoody@example.net,Individual,USA
7,37,Jackie Dixon,john23@example.org,SMB,USA
8,41,Nicholas Meyers,mendozaanne@example.com,Enterprise,USA
9,44,Stephen Cowan,cwright@example.com,Enterprise,USA


## Complex JOIN Queries

In [34]:
await ask_agent("How many orders has each customer segment placed?")

🤔 Question: How many orders has each customer segment placed?

📊 Results:


,segment,order_count
0,Enterprise,97
1,Individual,143
2,SMB,160


In [32]:
await ask_agent("Which customers have never placed an order?")

🤔 Question: Which customers have never placed an order?

📊 Results:


,name,email
0,Tracy Thomas,jwilson@example.org
1,Shawn Hughes,joy75@example.org
2,Colton Ramos,cassie24@example.net
3,Jackie Dixon,john23@example.org
4,Mark Salazar,amandaparker@example.net
5,Crystal Johnson,michael05@example.net
6,Erin Young,lmorales@example.org
7,Julie Cooper,kellylauren@example.net
8,Maria Neal,rjefferson@example.org
9,Kimberly Yates,jeffpayne@example.net


---

# Section 9: Error Handling and Best Practices

## Common Issues and Solutions

### Issue 1: Ambiguous Questions

**Bad:** "Show me the data" (What data? Which table?)

**Good:** "Show me all customers from the USA"

In [33]:
# This will likely generate better SQL
await ask_agent("Show me all customers from the USA")

🤔 Question: Show me all customers from the USA

📊 Results:


,id,name,email,segment,country,created_at,updated_at
0,1,Craig Robinson,cescobar@example.net,SMB,USA,2026-01-07 06:47:51.713204,2026-01-07 06:47:51.713204
1,10,Bryce Russell,timothymartin@example.net,Individual,USA,2026-01-07 06:47:51.713204,2026-01-07 06:47:51.713204
2,16,Rebecca Blevins,amandathomas@example.com,SMB,USA,2026-01-07 06:47:51.713204,2026-01-07 06:47:51.713204
3,21,Kathy Finley,robert43@example.org,Enterprise,USA,2026-01-07 06:47:51.713204,2026-01-07 06:47:51.713204
4,23,Megan Diaz,brittanyhouston@example.net,SMB,USA,2026-01-07 06:47:51.713204,2026-01-07 06:47:51.713204
5,27,Melanie King,qgrant@example.net,SMB,USA,2026-01-07 06:47:51.713204,2026-01-07 06:47:51.713204
6,30,Christopher Sparks,ymoody@example.net,Individual,USA,2026-01-07 06:47:51.713204,2026-01-07 06:47:51.713204
7,37,Jackie Dixon,john23@example.org,SMB,USA,2026-01-07 06:47:51.713204,2026-01-07 06:47:51.713204
8,41,Nicholas Meyers,mendozaanne@example.com,Enterprise,USA,2026-01-07 06:47:51.713204,2026-01-07 06:47:51.713204
9,44,Stephen Cowan,cwright@example.com,Enterprise,USA,2026-01-07 06:47:51.713204,2026-01-07 06:47:51.713204


### Issue 2: Incorrect Column Names

**Problem:** Agent looks for 'price' in orders table (doesn't exist)

**Solution:** Provide clear schema documentation (we do this in `SCHEMA_CONTEXT`)

In [35]:
# Schema context prevents this error
await ask_agent("What is the total revenue?")  # Will correctly use 'total_amount'

🤔 Question: What is the total revenue?

📊 Results:


,total_revenue
0,978331.65


### Issue 3: Case Sensitivity

**Problem:** PostgreSQL string comparisons are case-sensitive

**Solution:** Document exact values in schema context

In [36]:
# This works because we documented 'Delivered' (capitalized) in schema
await ask_agent("Show delivered orders")

🤔 Question: Show delivered orders

📊 Results:


,id,customer_name,order_date,total_amount,shipping_address
0,1,Pamela Holt,2025-04-12,4664.19,Unit 0084 Box 8448\nDPO AE 03858
1,3,Christopher Coleman,2025-05-09,1096.18,"51041 Margaret Stream\nSouth Erica, RI 61867"
2,4,William Hatfield,2025-06-01,4321.68,"94942 Mary Lodge Suite 686\nYolandaburgh, TN 8..."
3,5,Marcus Moyer,2025-12-17,738.59,"PSC 6888, Box 2931\nAPO AA 67668"
4,6,Angel Wilson,2025-04-05,3809.02,"27842 Brittany Greens Suite 245\nBeverlyland, ..."
...,...,...,...,...,...
280,395,Jason Taylor,2025-08-02,3256.59,"18376 Caitlin Shore\nLake Heatherland, NE 27487"
281,396,Ashley Green,2026-01-14,2381.09,"6701 Michael Circles\nNorth Mitchell, WA 72145"
282,397,James Nichols,2025-09-05,4881.87,None
283,398,Peter Thomas,2026-01-16,216.97,Unit 1207 Box 4672\nDPO AA 60733


## Best Practices

1. **Always provide schema context** - Critical for accuracy
2. **Be specific in questions** - Avoid ambiguous language
3. **Document exact values** - Include enum values, case sensitivity
4. **Verify results** - Compare Agent output with direct SQL
5. **Use descriptive column names** - Helps the LLM understand intent
6. **Include business logic** - Document relationships and constraints
7. **Test edge cases** - Empty results, NULLs, etc.
8. **Monitor and log** - Track query accuracy in production

## Production Considerations

1. **User Permissions** - Implement proper `UserResolver` with auth
2. **SQL Validation** - Block dangerous operations (DROP, DELETE without WHERE)
3. **Rate Limiting** - Prevent abuse
4. **Query Approval** - Show SQL to users before execution
5. **Monitoring** - Track failed queries for retraining
6. **Caching** - Cache common questions
7. **Row-Level Security** - Filter data based on user permissions
8. **API Key Management** - Never hardcode credentials

---

# Section 10: Next Steps and Resources

## What We've Learned

✅ Text-to-SQL fundamentals
✅ Vanna.ai 2.0 Agent architecture
✅ Setting up LLM, database, tools, and user resolver
✅ Importance of schema documentation
✅ Querying with natural language
✅ Handling async operations correctly
✅ Extracting results from UI components
✅ Best practices and error handling

## Key Takeaways

1. **Schema context is critical** - Without it, the Agent will generate incorrect SQL
2. **Vanna 2.0 uses async generators** - Use `async for` to iterate components
3. **UI components hold data in different attributes** - DataFrameComponent uses `rows`
4. **Direct SQL is useful** - For verification and simple queries
5. **Production requires more** - Permissions, validation, monitoring

## Advanced Topics to Explore

- **Custom Tools** - Create your own tools beyond RunSqlTool
- **Multi-Turn Conversations** - Agent memory and context
- **Different LLMs** - Try Claude, Llama, or other models
- **Different Databases** - MySQL, SQLite, Snowflake
- **Web Interface** - Vanna's `<vanna-chat>` component
- **Security** - Row-level security, query validation
- **Performance** - Caching, query optimization

## Resources

- **Vanna.ai Documentation**: https://docs.vanna.ai/
- **GitHub Repository**: https://github.com/vanna-ai/vanna
- **Examples**: https://github.com/vanna-ai/vanna/tree/main/examples
- **Community**: Discord, GitHub Discussions
- **API Reference**: https://docs.vanna.ai/api/

## Your Next Project Ideas

1. **Build a Slack Bot** - Answer data questions in Slack
2. **Create a Dashboard** - Visualize query results
3. **HR Analytics** - Query employee database
4. **Sales Intelligence** - Real-time sales queries
5. **Customer Support** - Automated data lookups

---

## Thank You!

You've completed the Vanna.ai Text-to-SQL tutorial. You now have the foundation to build your own Text-to-SQL applications!

**Happy coding! 🚀**